In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 27 02:19:06 2022

In [ ]:
@author: E5-471G
"""

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

set directory

In [ ]:
os.chdir('C:/Users/E5-471G/Documents/SIM-UOL/2021-22 Year 3/ST2195 Programming for Data Science/ST2195 Coursework')

load data

In [ ]:
airports = pd.read_csv('./airports.csv')
carriers = pd.read_csv('./carriers.csv')
planes = pd.read_csv('./plane-data.csv')

clean 'planes' and rename column 'tailnum'

In [ ]:
planes = planes.dropna()
planes = planes.rename(columns={'tailnum': 'TailNum'})

In [ ]:
ontime_2007 = pd.read_csv('./2007.csv.bz2', compression = 'bz2')
ontime_2008 = pd.read_csv('./2008.csv.bz2', compression = 'bz2')
ontime = pd.concat([ontime_2007, ontime_2008])

remove temporary dataframes

In [ ]:
del ontime_2007, ontime_2008

==== Q1 ====<br>
When is the best time of day, day of the week, and time of year to fly to minimise delays?

individually<br>
best month/time of year

In [ ]:
q1_month = ontime.loc[(ontime['Cancelled'] == 0) & (ontime['Diverted'] == 0)][['Month', 'DepDelay', 'ArrDelay']]
q1_month = q1_month.groupby(by = ['Month']).mean().round(2).sort_values(by = ['DepDelay', 'ArrDelay'], ascending = True)
print(q1_month)
# best month is the 9th month, September

plot histogram for q1_month

In [ ]:
plt.figure(figsize=(11,5))
plt.hist([q1_month.ArrDelay, q1_month.DepDelay], color = ['r','b'], label = ['Arrival Delay', 'Departure Delay'], alpha=0.5)
plt.xticks(range(17))
plt.title('Distribution of average delay per no. of months')
plt.ylabel('Frequency')
plt.xlabel('Average Delay (in mins)')
plt.legend()

best day of the week

In [ ]:
q1_day = ontime.loc[(ontime['Cancelled'] == 0) & (ontime['Diverted'] == 0)][['DayOfWeek', 'DepDelay', 'ArrDelay']]
q1_day = q1_day.groupby(by = ['DayOfWeek']).mean().round(2).sort_values(by = ['DepDelay', 'ArrDelay'], ascending = True)
print(q1_day)
# best day is the 6th day of the week, Saturday

plot histogram for q1_day

In [ ]:
plt.figure(figsize=(11,5))
plt.hist([q1_day.ArrDelay, q1_day.DepDelay], color = ['r','b'], label = ['Arrival Delay', 'Departure Delay'], alpha=0.5)
plt.xticks(range(15))
plt.title('Distribution of average delay by no. of days of the week')
plt.ylabel('Frequency')
plt.xlabel('Average Delay (in mins)')
plt.legend()
plt.show()

best time of the day

In [ ]:
q1_time = ontime.loc[(ontime['Cancelled'] == 0) & (ontime['Diverted'] == 0)][['CRSDepTime', 'DepDelay', 'ArrDelay']]
q1_time = q1_time.groupby(by = ['CRSDepTime']).mean().round(2).sort_values(by = ['DepDelay', 'ArrDelay'], ascending = True)
print(q1_time)
# best time is 3:15am (regardless of day, month)

best time of the day by month and day

In [ ]:
q1 = ontime.loc[(ontime['Cancelled'] == 0) & (ontime['Diverted'] == 0)][['CRSDepTime', 'DayOfWeek', 'Month', 'DepDelay', 'ArrDelay']]

In [ ]:
q1 = q1.groupby(by = ['Month', 'DayOfWeek'])['DepDelay', 'ArrDelay'].mean().round(2).sort_values(by = ['DepDelay', 'ArrDelay'], ascending = True)
print(q1)
# output is same as R's, but doesn't show CRSDepTime

R's corresponding output:<br>
best time by month and day is 11.30am<br>
best month, day, and time is November, Saturday, 11.30am

==== Q2 ====<br>
Do older planes suffer more delays?<br>
interpret qn as 'how many times did planes manufactured in xx year got delayed (arr/dep)'

combine ontime and planes dataframes

In [ ]:
data1 = ontime.merge(planes, on='TailNum', how='inner', suffixes=('_1', '_2'))

get 'year', 'DepDelay', and 'ArrDelay' where Cancelled & Diverted = 0, year value not equals to 0 (empty)

In [ ]:
q2 = data1.loc[(data1['Cancelled'] == 0) & (data1['Diverted'] == 0) & (data1['year'] != 0)][['year', 'DepDelay', 'ArrDelay']]
q2['overallNum_delays'] = q2['DepDelay'] + q2['ArrDelay']
q2 = q2.groupby(by=['year'])['overallNum_delays'].count().reset_index()
# check all 51 rows
q2.head(51)
# drop row 0 and row 50 where values are '0000' and 'None' respectively
q2 = q2.drop([0,50])
# check remaining 49 rows
q2.head(49)

plot barchart<br>
set the figure size

In [ ]:
plt.figure(figsize=(25, 10))
sns.set_style('whitegrid')
sns.barplot(
    y='overallNum_delays', 
    x='year', 
    data=q2, 
    ci=None, 
    color='#69b3a2')
plt.title("Total no. of departure and arrival delays by planes' year of manufacture in 2007 to 2008")
plt.ylabel("Number of Overall Delay")
plt.xlabel("Year of Manufacture of Planes")
plt.show()

check datatype of year, change to int

In [ ]:
q2.dtypes
q2['year'] = q2['year'].astype(int)
q2.dtypes

show percentage by half of total year count<br>
2008-1956 = 52, 52/2 = 26, 2008-26 = 1982

sum counts for planes aged 26 years and younger (>=1982) & planes older than 26 years (<1982)

In [ ]:
q2_c1 = q2.loc[q2['year'] < 1982, 'overallNum_delays'].sum()
q2_c2 = q2.loc[q2['year'] >= 1982, 'overallNum_delays'].sum()

percentage of counts for years <= 1982 & > 1982<br>
create dictionary then dataframe

In [ ]:
q2_p = {'year': ['1956-1981', '1982-2008'], 'count': [q2_c1, q2_c2]}
q2_p = pd.DataFrame(q2_p, columns = ['year', 'count'])
q2_p['percentage'] = (q2_p['count']/q2_p['count'].sum()) * 100
q2_p.round(2)

piechart

In [ ]:
plt.pie(q2_p['count'], labels = q2_p['year'], labeldistance=1.15, wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'white' })
plt.show()

==== Q3 ====<br>
How does the number of people flying between different locations change over time?

rename airports column 'iata' to 'Dest'

In [ ]:
airports1 = airports.rename(columns={'iata': 'Dest'})

combine ontime and airports1 dataframes on 'Dest'

In [ ]:
data2 = ontime.merge(airports1, on='Dest', how='inner', suffixes=('_1', '_2'))

get total inbound flights (with no cancelled flights) by airport, city, state per month per year

In [ ]:
q3 = data2.loc[(data2['Cancelled'] == 0)][['Year', 'Month', 'airport', 'city', 'state', 'Dest']]
q3 = q3.groupby(by=['Year', 'Month', 'airport', 'city', 'state'])['Dest'].count().reset_index().sort_values(['Year', 'Month', 'Dest'], ascending = [True, True, False])

show top 10 airports by total inbound flights per month per year

In [ ]:
q3 = q3.groupby(by=['Year','Month']).head(10).reset_index(drop=True)
q3.head(20)

show seasonality using lineplot

In [ ]:
plt.figure(figsize=(25, 10))
sns.lineplot(q3['Month'], q3['Dest'], hue=q3['Year'], palette=["C0", "C1"])
plt.title("Seasonality of Top 10 Total Inbound Flights in 2007 to 2008")
plt.ylabel("Total Inbound Flights")
plt.xlabel("Month")
plt.xlim([1,12])
plt.grid
plt.show

==== Q4 ====<br>
Can you detect cascading failures as delays in one airport create delays in others?

interpret question as 'regardless of aircrafts departing on time/late from airport A/B, do they arrive late on schedule to airports B/C...'<br>
late aircraft delay refers to late arriving aircrafts<br>
from airport A to B via C -- A to C, C to B, or more stops<br>
query for same day, same flight aircrafts that departed late with LateAircraftDelay > 60 mins

rename airports column 'iata' to 'Origin'

In [ ]:
airports2 = airports.rename(columns={'iata': 'Origin'})

combine ontime and airports2 dataframes on 'Origin'

In [ ]:
data3 = ontime.merge(airports2, on='Origin', how='inner', suffixes=('_1', '_2'))

get year, month, day, flight no., tail no., origin, dest, depdelay and lateaircraft delay <br>
where Cancelled & Diverted = 0, DepDelay >= 0, LateAircraftDelay > 0

In [ ]:
q4 = data3.loc[(data3['Cancelled'] == 0) & (data3['Diverted']==0) & (data3['DepDelay']>=0) & (data3['LateAircraftDelay']>0)][['Year', 'Month', 'DayofMonth', 'FlightNum', 'TailNum', 'Origin', 'Dest', 'DepDelay', 'LateAircraftDelay']]
q4 = q4.sort_values(['Year', 'Month', 'DayofMonth','FlightNum', 'TailNum'], ascending = True)
q4 = q4[q4.duplicated(['Year', 'Month', 'DayofMonth', 'FlightNum', 'TailNum'], keep = False)]
q4.head(20)

same output as R<br>
yes, can detect

==== Q5 ====<br>
Use the available variables to construct a model that predicts delays.

response/dependent variable is delays (in mins) - continuous, use regression<br>
do linear, lasso, and ridge<br>
compare metrics

import libraries for regression

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

create dataframe for modelling delays<br>
let 'delays' refer to arrival delay + departure delay, ie. overall delay (in mins)<br>
arrival delay = carrier + weather + NAS + security + late aircraft delays 

In [ ]:
q5 = ontime.loc[(ontime['Cancelled'] == 0) & (ontime['Diverted'] == 0)][['DepDelay', 'CRSDepTime', 'ArrDelay', 'CRSArrTime', 'AirTime', 'Distance', 'TaxiIn', 'TaxiOut']]
q5['overallDelay'] = q5['DepDelay'] + q5['ArrDelay']
q5.head()

check for NA

In [ ]:
q5.describe()

limit sample size to 1000, selected randomly (results may differ)

In [ ]:
q5 = q5.sample(1000)

create features and response variable

In [ ]:
target_column = ['overallDelay'] 
predictors = list(set(list(q5.columns))-set(target_column))
q5[predictors] = q5[predictors]/q5[predictors].max()
q5.describe()

create train and test sets

In [ ]:
x = q5[predictors].values
y = q5[target_column].values

split train 70%, test 30%

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)
print(x_train.shape)
print(x_test.shape)

== linear regression ==

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train) # fit model

predict MAE, MSE, RMSE & R-Squared

In [ ]:
y_pred_lr = lr.predict(x_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_lr))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_lr))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_lr))) 
print('R-squared:', r2_score(y_test, y_pred_lr))

In [ ]:
print('Intercept:', lr.intercept_)
print('Coefficient', lr.coef_)

linear model equation changes per run due to random sample selection<br>
overallDelay = -2.84217094e-14 -2.66707737e-13(DepDelay) +5.86197757e-14(CRSDepDelay) -2.62012634e-14(ArrDelay) +6.85000000e+02(CRSArrDelay) -3.52495810e-15(AirTime) +1.65201186e-13(Distance) +2.54463117e-13 (TaxiIn) +6.90000000e+02(TaxiOut)

plot linear regression

In [ ]:
plt.figure(figsize=(12,8))
sns.regplot(x=y_pred_lr, y=y_test, marker='o', line_kws={"color":"r","alpha":0.6,"lw":2}, scatter_kws={'s':5})
plt.xlabel("Predicted Overall Delay (in mins)")
plt.ylabel("Actual Overall Delay (in mins)")
plt.title("Linear Regression Model")
plt.show()

== ridge regression ==

In [ ]:
rr = Ridge(alpha=0.01)
rr.fit(x_train, y_train)

In [ ]:
y_pred_rr = rr.predict(x_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_rr))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_rr))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_rr))) 
print('R-squared:', r2_score(y_test, y_pred_rr))

find optimal alpha using RidgeCV

In [ ]:
model_rr = RidgeCV(alphas=[0.1, 1.0, 10.0]).fit(x_train, y_train)
model_rr.alpha_ 
# 0.1

with optimal ridge alpha

In [ ]:
rr_best = Ridge(alpha = model_rr.alpha_)
rr_best.fit(x_train, y_train)

In [ ]:
y_pred_rr_best = rr_best.predict(x_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_rr_best))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_rr_best))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_rr_best))) 
print('R-squared:', r2_score(y_test, y_pred_rr_best))

== lasso regression ==

In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(x_train, y_train)

In [ ]:
y_pred_lasso = lasso.predict(x_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_lasso))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_lasso))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_lasso))) 
print('R-squared:', r2_score(y_test, y_pred_lasso))

find optimal alpha using LassoCV<br>
lasso with 10 fold cross-validation

In [ ]:
model_lasso = LassoCV(cv=10, random_state=0, max_iter=10000)
model_lasso.fit(x_train, y_train)
model_lasso.alpha_ 
# 0.002171072304483836 (will change depending on run)

with optimal lasso alpha

In [ ]:
lasso_best = Ridge(alpha = model_lasso.alpha_)
lasso_best.fit(x_train, y_train)

In [ ]:
y_pred_lasso_best = lasso_best.predict(x_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_lasso_best))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_lasso_best))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_lasso_best))) 
print('R-squared:', r2_score(y_test, y_pred_lasso_best))

create dataframe to compare models

In [ ]:
comparison = pd.DataFrame({'Linear': [mean_absolute_error(y_test, y_pred_lr), mean_squared_error(y_test, y_pred_lr), np.sqrt(mean_squared_error(y_test, y_pred_lr)), r2_score(y_test, y_pred_lr)], 'Ridge(alpha=0.01)': [mean_absolute_error(y_test, y_pred_rr), mean_squared_error(y_test, y_pred_rr), np.sqrt(mean_squared_error(y_test, y_pred_rr)), r2_score(y_test, y_pred_rr)], 'Ridge(optimal alpha)': [mean_absolute_error(y_test, y_pred_rr_best), mean_squared_error(y_test, y_pred_rr_best), np.sqrt(mean_squared_error(y_test, y_pred_rr_best)), r2_score(y_test, y_pred_rr_best)], 'Lasso(alpha=0.01)': [mean_absolute_error(y_test, y_pred_lasso), mean_squared_error(y_test, y_pred_lasso), np.sqrt(mean_squared_error(y_test, y_pred_lasso)), r2_score(y_test, y_pred_lasso)], 'Lasso(optimal alpha)': [mean_absolute_error(y_test, y_pred_lasso_best), mean_squared_error(y_test, y_pred_lasso_best), np.sqrt(mean_squared_error(y_test, y_pred_lasso_best)), r2_score(y_test, y_pred_lasso_best)]})
comparison.index = ['MAE', 'MSE', 'RMSE', 'R-squared']
# allow console to display max rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# show float to 15 decimal places to account for linear 
pd.set_option('display.float_format', '{:.15f}'.format)
comparison